In [1]:
import numpy as np
import pandas as pd
from datetime import timedelta
import glob
import os

import warnings
warnings.filterwarnings(action='ignore')

In [3]:
data = pd.read_csv('data/서울특별시 집단급식소 인허가 정보_v2.csv', encoding='cp949')
data

,인허가일자,상세영업상태명,폐업일자,주소,사업장명,업태구분명
0,20200714,영업,29991231,서울특별시 마포구 대흥동 17-5 신촌그랑자이,신촌그랑어린이집,어린이집
1,20200714,영업,29991231,서울특별시 은평구 응암동 626-53 백련산파크자이,구립한빛어린이집,어린이집
2,20170303,영업,29991231,서울특별시 강서구 마곡동 798-0 홈앤쇼핑사옥 (지상 11층),직원식당,기숙사
3,20130227,폐업,20200714,서울특별시 구로구 구로동 43-4 다솜아파트 110호 111호,다솜어린이집,어린이집
4,20050824,폐업,20200714,서울특별시 동작구 사당동 190-171,송림유치원,어린이집
...,...,...,...,...,...,...
7479,20200925,영업,29991231,서울특별시 영등포구 대림동 917-49 e편한세상 영등포 아델포레,새솔아델포레어린이집,어린이집
7480,20200925,영업,29991231,서울특별시 성동구 금호동4가 340 서울숲푸르지오아파트,주식회사 제스아일랜드교육,기타 집단급식소
7481,20120709,영업,29991231,서울특별시 은평구 녹번동 152-79 우리어린이집,우리어린이집,어린이집
7482,20050620,폐업,20200925,서울특별시 광진구 자양동 16-3,파랑어린이집,어린이집


In [4]:
data['업태구분명'].unique()

array(['어린이집', '기숙사', '병원', '학교', '기타 집단급식소', '산업체', '사회복지시설', '공공기관',
       '산후조리원', '집단급식소', '수련원'], dtype=object)

In [106]:
# 영업의 폐업일자를 0으로 변경
now_open = data[data['상세영업상태명']=='영업'].reset_index(drop=True)
now_open['폐업일자']=0
now_close = data[data['상세영업상태명']=='폐업'].reset_index(drop=True)
data = pd.concat([now_open, now_close], axis=0).reset_index(drop=True)
data

,인허가일자,상세영업상태명,폐업일자,주소,사업장명,업태구분명
0,20200714,영업,0,서울특별시 마포구 대흥동 17-5 신촌그랑자이,신촌그랑어린이집,어린이집
1,20200714,영업,0,서울특별시 은평구 응암동 626-53 백련산파크자이,구립한빛어린이집,어린이집
2,20170303,영업,0,서울특별시 강서구 마곡동 798-0 홈앤쇼핑사옥 (지상 11층),직원식당,기숙사
3,20050707,영업,0,서울특별시 강서구 내발산동 746-0 사무엘어린이집,사무엘어린이집,어린이집
4,20200715,영업,0,서울특별시 영등포구 대림동 603-26 코스모메디칼,새롬한방병원,병원
...,...,...,...,...,...,...
7479,20091216,폐업,20200902,서울특별시 광진구 중곡동 134-26,한울어린이집,어린이집
7480,20160408,폐업,20200904,서울특별시 구로구 구로동 787-70 재단법인 한국천주교살레시오회 요셉어린이집,재단법인 한국천주교살레시오회 요셉어린이집,어린이집
7481,20150429,폐업,20200908,서울특별시 송파구 가락동 16-12 동일시티빌 지상1층,베베궁 음악학원,산업체
7482,20141205,폐업,20200921,서울특별시 노원구 중계동 511-2 중계주공4단지상가동 2층 205호,노원구립장애인일자리센터,공공기관


### 1. 어린이집

In [107]:
# 어린이집 데이터 분리
pre_school = data[data['사업장명'].str.contains('어린이집')]
pre = data[(data['업태구분명']=='어린이집')&(data['사업장명'].str.contains('어린이집')==False)&(data['사업장명'].str.contains('유치원')==False)]
df = pd.concat([pre_school, pre], axis=0).reset_index(drop=True)
df

,인허가일자,상세영업상태명,폐업일자,주소,사업장명,업태구분명
0,20200714,영업,0,서울특별시 마포구 대흥동 17-5 신촌그랑자이,신촌그랑어린이집,어린이집
1,20200714,영업,0,서울특별시 은평구 응암동 626-53 백련산파크자이,구립한빛어린이집,어린이집
2,20050707,영업,0,서울특별시 강서구 내발산동 746-0 사무엘어린이집,사무엘어린이집,어린이집
3,20130726,영업,0,서울특별시 영등포구 문래동3가 55-18 영시티,하람어린이집,어린이집
4,20200716,영업,0,서울특별시 은평구 응암동 36 녹번역e편한세상캐슬아파트,구립응암행복어린이집,어린이집
...,...,...,...,...,...,...
2723,20050704,영업,0,서울특별시 구로구 고척동 334 벽산블루밍아파트 관리동 1층,미키몬테소리,어린이집
2724,20050713,폐업,20200324,서울특별시 서대문구 북가좌동 124-12번지 지하1층,알티오라어학원,어린이집
2725,20180417,폐업,20190520,서울특별시 양천구 신정동 1277번지 신트리4단지아파트 보육원동,혜원유아학교,어린이집
2726,20050415,폐업,20200710,서울특별시 강동구 길동 399-4,(주)예크트리,어린이집


In [108]:
# 업태구분명 : 어린이집으로 변경
df.drop('업태구분명', axis=1, inplace=True)
df['업태구분명']='어린이집'
df

,인허가일자,상세영업상태명,폐업일자,주소,사업장명,업태구분명
0,20200714,영업,0,서울특별시 마포구 대흥동 17-5 신촌그랑자이,신촌그랑어린이집,어린이집
1,20200714,영업,0,서울특별시 은평구 응암동 626-53 백련산파크자이,구립한빛어린이집,어린이집
2,20050707,영업,0,서울특별시 강서구 내발산동 746-0 사무엘어린이집,사무엘어린이집,어린이집
3,20130726,영업,0,서울특별시 영등포구 문래동3가 55-18 영시티,하람어린이집,어린이집
4,20200716,영업,0,서울특별시 은평구 응암동 36 녹번역e편한세상캐슬아파트,구립응암행복어린이집,어린이집
...,...,...,...,...,...,...
2723,20050704,영업,0,서울특별시 구로구 고척동 334 벽산블루밍아파트 관리동 1층,미키몬테소리,어린이집
2724,20050713,폐업,20200324,서울특별시 서대문구 북가좌동 124-12번지 지하1층,알티오라어학원,어린이집
2725,20180417,폐업,20190520,서울특별시 양천구 신정동 1277번지 신트리4단지아파트 보육원동,혜원유아학교,어린이집
2726,20050415,폐업,20200710,서울특별시 강동구 길동 399-4,(주)예크트리,어린이집


### 2. 유치원

In [109]:
# 유치원 데이터 분리
df2 = data[data['사업장명'].str.contains('유치원')].reset_index(drop=True)
df2

,인허가일자,상세영업상태명,폐업일자,주소,사업장명,업태구분명
0,20170517,영업,0,서울특별시 성북구 보문동4가 4,동선유치원,학교
1,20050415,영업,0,서울특별시 강동구 천호동 302-10,오송유치원,어린이집
2,20050415,영업,0,서울특별시 강동구 천호동 397-413,(재)천주교서울대교구유지재단 천호유치원,어린이집
3,20200714,영업,0,서울특별시 동대문구 답십리동 36-32,라온유치원,어린이집
4,20200407,영업,0,서울특별시 관악구 봉천동 1717-2번지 은비유치원,서울청림유치원,어린이집
...,...,...,...,...,...,...
702,20081203,폐업,20200312,서울특별시 노원구 상계동 720 주공아파트 6단지내,영산유치원,어린이집
703,20051021,폐업,20200729,서울특별시 관악구 신림동 1433-55,신지유치원,어린이집
704,20161025,폐업,20200803,서울특별시 성북구 돈암동 609,우촌유치원,어린이집
705,20070904,폐업,20200812,서울특별시 노원구 중계동 511-2 상원유치원,상원유치원,어린이집


In [110]:
# 업태구분명 : 유치원으로 변경
df2.drop('업태구분명', axis=1, inplace=True)
df2['업태구분명'] = '유치원'
df2

,인허가일자,상세영업상태명,폐업일자,주소,사업장명,업태구분명
0,20170517,영업,0,서울특별시 성북구 보문동4가 4,동선유치원,유치원
1,20050415,영업,0,서울특별시 강동구 천호동 302-10,오송유치원,유치원
2,20050415,영업,0,서울특별시 강동구 천호동 397-413,(재)천주교서울대교구유지재단 천호유치원,유치원
3,20200714,영업,0,서울특별시 동대문구 답십리동 36-32,라온유치원,유치원
4,20200407,영업,0,서울특별시 관악구 봉천동 1717-2번지 은비유치원,서울청림유치원,유치원
...,...,...,...,...,...,...
702,20081203,폐업,20200312,서울특별시 노원구 상계동 720 주공아파트 6단지내,영산유치원,유치원
703,20051021,폐업,20200729,서울특별시 관악구 신림동 1433-55,신지유치원,유치원
704,20161025,폐업,20200803,서울특별시 성북구 돈암동 609,우촌유치원,유치원
705,20070904,폐업,20200812,서울특별시 노원구 중계동 511-2 상원유치원,상원유치원,유치원


### 3.  2017~2019, 2020 데이터 생성

In [111]:
result_df = pd.concat([df, df2], axis=0).reset_index(drop=True)
result_df

,인허가일자,상세영업상태명,폐업일자,주소,사업장명,업태구분명
0,20200714,영업,0,서울특별시 마포구 대흥동 17-5 신촌그랑자이,신촌그랑어린이집,어린이집
1,20200714,영업,0,서울특별시 은평구 응암동 626-53 백련산파크자이,구립한빛어린이집,어린이집
2,20050707,영업,0,서울특별시 강서구 내발산동 746-0 사무엘어린이집,사무엘어린이집,어린이집
3,20130726,영업,0,서울특별시 영등포구 문래동3가 55-18 영시티,하람어린이집,어린이집
4,20200716,영업,0,서울특별시 은평구 응암동 36 녹번역e편한세상캐슬아파트,구립응암행복어린이집,어린이집
...,...,...,...,...,...,...
3430,20081203,폐업,20200312,서울특별시 노원구 상계동 720 주공아파트 6단지내,영산유치원,유치원
3431,20051021,폐업,20200729,서울특별시 관악구 신림동 1433-55,신지유치원,유치원
3432,20161025,폐업,20200803,서울특별시 성북구 돈암동 609,우촌유치원,유치원
3433,20070904,폐업,20200812,서울특별시 노원구 중계동 511-2 상원유치원,상원유치원,유치원


In [112]:
# 2017년 데이터
df_2017 = result_df[(result_df['인허가일자']<20180101)].reset_index(drop=True)
df_2017['년'] = 2017
df_2017

,인허가일자,상세영업상태명,폐업일자,주소,사업장명,업태구분명,년
0,20050707,영업,0,서울특별시 강서구 내발산동 746-0 사무엘어린이집,사무엘어린이집,어린이집,2017
1,20130726,영업,0,서울특별시 영등포구 문래동3가 55-18 영시티,하람어린이집,어린이집,2017
2,20050201,영업,0,서울특별시 강동구 천호동 340-39,구립빗살어린이집,어린이집,2017
3,20171208,영업,0,서울특별시 강서구 마곡동 748번지 마곡13단지힐스테이트마스터 (지상 1층),마곡13하나어린이집,어린이집,2017
4,20160527,영업,0,서울특별시 강서구 내발산동 740-1 (발산1동),하나린어린이집,어린이집,2017
...,...,...,...,...,...,...,...
2874,20081203,폐업,20200312,서울특별시 노원구 상계동 720 주공아파트 6단지내,영산유치원,유치원,2017
2875,20051021,폐업,20200729,서울특별시 관악구 신림동 1433-55,신지유치원,유치원,2017
2876,20161025,폐업,20200803,서울특별시 성북구 돈암동 609,우촌유치원,유치원,2017
2877,20070904,폐업,20200812,서울특별시 노원구 중계동 511-2 상원유치원,상원유치원,유치원,2017


In [117]:
# 영업, 폐업 구분
## open_df : 2017년에 영업
## closed_df : 2017년에 폐업
open_df = df_2017[(df_2017['폐업일자'] >= 20180101) | (df_2017['폐업일자'] == 0)].reset_index(drop=True)
closed_df = df_2017[(df_2017['폐업일자'] < 20180101)&(df_2017['폐업일자'] != 0)].reset_index(drop=True)
closed_df

,인허가일자,상세영업상태명,폐업일자,주소,사업장명,업태구분명,년
0,20160308,폐업,20170427,서울특별시 종로구 궁정동 4-11번지 2층,무궁화어린이집,어린이집,2017
1,20140203,폐업,20170124,서울특별시 종로구 신문로2가 1-158번지 지상1층,김&장 (KIM&CHANG) 어린이집,어린이집,2017
2,20160120,폐업,20170315,서울특별시 종로구 신문로2가 1-183번지 지상12층,교보생명 다솜이 어린이집,어린이집,2017
3,20160317,폐업,20170607,서울특별시 중구 남대문로2가 5-3번지,SK네트웍스 새싹어린이집,어린이집,2017
4,20120502,폐업,20170228,서울특별시 중구 을지로1가 50번지 삼성화재빌딩 4층,삼성화재을지로어린이집,어린이집,2017
...,...,...,...,...,...,...,...
132,20090814,폐업,20170529,서울특별시 영등포구 여의도동 53-13번지,침례유치원,유치원,2017
133,20081013,폐업,20170209,서울특별시 동작구 신대방동 385-2번지,태성유치원,유치원,2017
134,20050912,폐업,20170216,서울특별시 동작구 노량진동 323번지 우성아파트 내,장아유치원,유치원,2017
135,20040227,폐업,20170310,서울특별시 서초구 잠원동 70-3번지 2층,한신유치원,유치원,2017


In [118]:
open_df['폐업일자'] = 0
open_df['상세영업상태명'] = '영업'
open_df

,인허가일자,상세영업상태명,폐업일자,주소,사업장명,업태구분명,년
0,20050707,영업,0,서울특별시 강서구 내발산동 746-0 사무엘어린이집,사무엘어린이집,어린이집,2017
1,20130726,영업,0,서울특별시 영등포구 문래동3가 55-18 영시티,하람어린이집,어린이집,2017
2,20050201,영업,0,서울특별시 강동구 천호동 340-39,구립빗살어린이집,어린이집,2017
3,20171208,영업,0,서울특별시 강서구 마곡동 748번지 마곡13단지힐스테이트마스터 (지상 1층),마곡13하나어린이집,어린이집,2017
4,20160527,영업,0,서울특별시 강서구 내발산동 740-1 (발산1동),하나린어린이집,어린이집,2017
...,...,...,...,...,...,...,...
2737,20081203,영업,0,서울특별시 노원구 상계동 720 주공아파트 6단지내,영산유치원,유치원,2017
2738,20051021,영업,0,서울특별시 관악구 신림동 1433-55,신지유치원,유치원,2017
2739,20161025,영업,0,서울특별시 성북구 돈암동 609,우촌유치원,유치원,2017
2740,20070904,영업,0,서울특별시 노원구 중계동 511-2 상원유치원,상원유치원,유치원,2017


In [119]:
fin_2017_df = pd.concat([open_df, closed_df], axis=0).reset_index(drop=True)
fin_2017_df.sort_values(by=['인허가일자'], axis=0, inplace=True)
fin_2017_df.reset_index(drop=True, inplace=True)

In [120]:
# 2018년 데이터
df_2018 = result_df[(result_df['인허가일자']<20190101) & (result_df['폐업일자']>=20180101) | (result_df['폐업일자']==0)].reset_index(drop=True)
df_2018['년'] = 2018
df_2018

,인허가일자,상세영업상태명,폐업일자,주소,사업장명,업태구분명,년
0,20200714,영업,0,서울특별시 마포구 대흥동 17-5 신촌그랑자이,신촌그랑어린이집,어린이집,2018
1,20200714,영업,0,서울특별시 은평구 응암동 626-53 백련산파크자이,구립한빛어린이집,어린이집,2018
2,20050707,영업,0,서울특별시 강서구 내발산동 746-0 사무엘어린이집,사무엘어린이집,어린이집,2018
3,20130726,영업,0,서울특별시 영등포구 문래동3가 55-18 영시티,하람어린이집,어린이집,2018
4,20200716,영업,0,서울특별시 은평구 응암동 36 녹번역e편한세상캐슬아파트,구립응암행복어린이집,어린이집,2018
...,...,...,...,...,...,...,...
3277,20081203,폐업,20200312,서울특별시 노원구 상계동 720 주공아파트 6단지내,영산유치원,유치원,2018
3278,20051021,폐업,20200729,서울특별시 관악구 신림동 1433-55,신지유치원,유치원,2018
3279,20161025,폐업,20200803,서울특별시 성북구 돈암동 609,우촌유치원,유치원,2018
3280,20070904,폐업,20200812,서울특별시 노원구 중계동 511-2 상원유치원,상원유치원,유치원,2018


In [121]:
# 영업, 폐업 구분
## open_df : 2018년에 영업
## closed_df : 2018년에 폐업
open_df = df_2018[(df_2018['폐업일자'] >= 20190101) | (df_2018['폐업일자'] == 0)].reset_index(drop=True)
closed_df = df_2018[(df_2018['폐업일자'] < 20190101)&(df_2018['폐업일자'] != 0)].reset_index(drop=True)
print(len(df_2018), len(open_df)+len(closed_df))
closed_df

3282 3282


,인허가일자,상세영업상태명,폐업일자,주소,사업장명,업태구분명,년
0,20160307,폐업,20180328,서울특별시 종로구 신문로2가 1-124번지,현대해상 온마음 어린이집,어린이집,2018
1,20160418,폐업,20181231,서울특별시 종로구 창성동 117-6번지 1층,서울청사 한빛어린이집,어린이집,2018
2,20150313,폐업,20180327,서울특별시 종로구 관훈동 192-18번지 지상2층,SK건설 행복날개 어린이집,어린이집,2018
3,20160809,폐업,20181213,서울특별시 종로구 서린동 136번지 2층,kids-sure 어린이집,어린이집,2018
4,20070726,폐업,20181018,서울특별시 종로구 연건동 113-2번지 1층호,서울대학교병원어린이집,어린이집,2018
...,...,...,...,...,...,...,...
180,20090825,폐업,20180809,서울특별시 영등포구 영등포동8가 70번지,한영유치원,유치원,2018
181,20150909,폐업,20180502,서울특별시 관악구 신림동 412-234번지,송이유치원,유치원,2018
182,20050715,폐업,20180312,서울특별시 관악구 봉천동 893-23번지,남서울유치원,유치원,2018
183,20160318,폐업,20180223,서울특별시 강남구 개포동 188번지,서울개포유치원,유치원,2018


In [122]:
open_df['폐업일자'] = 0
open_df['상세영업상태명'] = '영업'
open_df

,인허가일자,상세영업상태명,폐업일자,주소,사업장명,업태구분명,년
0,20200714,영업,0,서울특별시 마포구 대흥동 17-5 신촌그랑자이,신촌그랑어린이집,어린이집,2018
1,20200714,영업,0,서울특별시 은평구 응암동 626-53 백련산파크자이,구립한빛어린이집,어린이집,2018
2,20050707,영업,0,서울특별시 강서구 내발산동 746-0 사무엘어린이집,사무엘어린이집,어린이집,2018
3,20130726,영업,0,서울특별시 영등포구 문래동3가 55-18 영시티,하람어린이집,어린이집,2018
4,20200716,영업,0,서울특별시 은평구 응암동 36 녹번역e편한세상캐슬아파트,구립응암행복어린이집,어린이집,2018
...,...,...,...,...,...,...,...
3092,20081203,영업,0,서울특별시 노원구 상계동 720 주공아파트 6단지내,영산유치원,유치원,2018
3093,20051021,영업,0,서울특별시 관악구 신림동 1433-55,신지유치원,유치원,2018
3094,20161025,영업,0,서울특별시 성북구 돈암동 609,우촌유치원,유치원,2018
3095,20070904,영업,0,서울특별시 노원구 중계동 511-2 상원유치원,상원유치원,유치원,2018


In [125]:
fin_2018_df = pd.concat([open_df, closed_df], axis=0).reset_index(drop=True)
fin_2018_df.sort_values(by=['인허가일자'], axis=0, inplace=True)
fin_2018_df.reset_index(drop=True, inplace=True)

In [128]:
# 2019년 데이터
df_2019 = result_df[(result_df['인허가일자']<20200101) & (result_df['폐업일자']>=20190101) | (result_df['폐업일자']==0)].reset_index(drop=True)
df_2019['년'] = 2019
df_2019

,인허가일자,상세영업상태명,폐업일자,주소,사업장명,업태구분명,년
0,20200714,영업,0,서울특별시 마포구 대흥동 17-5 신촌그랑자이,신촌그랑어린이집,어린이집,2019
1,20200714,영업,0,서울특별시 은평구 응암동 626-53 백련산파크자이,구립한빛어린이집,어린이집,2019
2,20050707,영업,0,서울특별시 강서구 내발산동 746-0 사무엘어린이집,사무엘어린이집,어린이집,2019
3,20130726,영업,0,서울특별시 영등포구 문래동3가 55-18 영시티,하람어린이집,어린이집,2019
4,20200716,영업,0,서울특별시 은평구 응암동 36 녹번역e편한세상캐슬아파트,구립응암행복어린이집,어린이집,2019
...,...,...,...,...,...,...,...
3104,20081203,폐업,20200312,서울특별시 노원구 상계동 720 주공아파트 6단지내,영산유치원,유치원,2019
3105,20051021,폐업,20200729,서울특별시 관악구 신림동 1433-55,신지유치원,유치원,2019
3106,20161025,폐업,20200803,서울특별시 성북구 돈암동 609,우촌유치원,유치원,2019
3107,20070904,폐업,20200812,서울특별시 노원구 중계동 511-2 상원유치원,상원유치원,유치원,2019


In [129]:
# 영업, 폐업 구분
## open_df : 2019년에 영업
## closed_df : 2019년에 폐업
open_df = df_2019[(df_2019['폐업일자'] >= 20200101) | (df_2019['폐업일자'] == 0)].reset_index(drop=True)
closed_df = df_2019[(df_2019['폐업일자'] < 20200101)&(df_2019['폐업일자'] != 0)].reset_index(drop=True)
print(len(df_2019), len(open_df)+len(closed_df))
closed_df

3109 3109


,인허가일자,상세영업상태명,폐업일자,주소,사업장명,업태구분명,년
0,20190716,폐업,20191119,서울특별시 종로구 동숭동 25-1번지 일석 기념관,서울대학교병원어린이집,어린이집,2019
1,20181018,폐업,20190716,서울특별시 종로구 동숭동 25-1번지 일석 기념관,서울대학병원 어린이집,어린이집,2019
2,20160427,폐업,20190405,서울특별시 종로구 재동 32-10번지 1층,롯데백화점 어린이집,어린이집,2019
3,20170427,폐업,20190313,서울특별시 종로구 궁정동 4-11번지 2층,무궁화 어린이집,어린이집,2019
4,20050810,폐업,20190612,서울특별시 종로구 신문로1가 42번지 언더우드기념관 1층,새문안어린이집,어린이집,2019
...,...,...,...,...,...,...,...
214,20050714,폐업,20190924,서울특별시 관악구 신림동 1694번지,하늘유치원,유치원,2019
215,20081231,폐업,20190321,서울특별시 강남구 역삼동 784-3번지 지하1층,청운교회부설유치원,유치원,2019
216,20090105,폐업,20190418,서울특별시 강남구 도곡동 963-0번지,럭키유치원,유치원,2019
217,20051226,폐업,20190515,서울특별시 송파구 오금동 53-12번지,그림유치원,유치원,2019


In [130]:
open_df['폐업일자'] = 0
open_df['상세영업상태명'] = '영업'
open_df

,인허가일자,상세영업상태명,폐업일자,주소,사업장명,업태구분명,년
0,20200714,영업,0,서울특별시 마포구 대흥동 17-5 신촌그랑자이,신촌그랑어린이집,어린이집,2019
1,20200714,영업,0,서울특별시 은평구 응암동 626-53 백련산파크자이,구립한빛어린이집,어린이집,2019
2,20050707,영업,0,서울특별시 강서구 내발산동 746-0 사무엘어린이집,사무엘어린이집,어린이집,2019
3,20130726,영업,0,서울특별시 영등포구 문래동3가 55-18 영시티,하람어린이집,어린이집,2019
4,20200716,영업,0,서울특별시 은평구 응암동 36 녹번역e편한세상캐슬아파트,구립응암행복어린이집,어린이집,2019
...,...,...,...,...,...,...,...
2885,20081203,영업,0,서울특별시 노원구 상계동 720 주공아파트 6단지내,영산유치원,유치원,2019
2886,20051021,영업,0,서울특별시 관악구 신림동 1433-55,신지유치원,유치원,2019
2887,20161025,영업,0,서울특별시 성북구 돈암동 609,우촌유치원,유치원,2019
2888,20070904,영업,0,서울특별시 노원구 중계동 511-2 상원유치원,상원유치원,유치원,2019


In [131]:
fin_2019_df = pd.concat([open_df, closed_df], axis=0).reset_index(drop=True)
fin_2019_df.sort_values(by=['인허가일자'], axis=0, inplace=True)
fin_2019_df.reset_index(drop=True, inplace=True)

In [140]:
# 2017~2019 데이터 병합
fin_df = pd.concat([fin_2017_df, fin_2018_df, fin_2019_df], axis=0).reset_index(drop=True)
fin_df = fin_df[['년', '인허가일자', '상세영업상태명', '폐업일자', '업태구분명', '사업장명', '주소']]
fin_df

,년,인허가일자,상세영업상태명,폐업일자,업태구분명,사업장명,주소
0,2017,19950120,영업,0,어린이집,개미어린이집,서울특별시 금천구 독산동 993-30번지 개미어린이집
1,2017,19950121,영업,0,어린이집,정심어린이집,서울특별시 금천구 독산동 938-5번지
2,2017,19950122,영업,0,어린이집,독산1동어린이집,서울특별시 금천구 독산동 999-11번지 독산1동어린이집
3,2017,19950123,영업,0,어린이집,새길어린이집,서울특별시 금천구 시흥동 산 13-8번지
4,2017,19950124,영업,0,어린이집,가산어린이집,서울특별시 금천구 가산동 602-71번지 가산어린이집상록경록당
...,...,...,...,...,...,...,...
9265,2019,20200915,영업,0,어린이집,지니어스어린이집,서울특별시 강서구 등촌동 637 서울시니어스가양타워(지상 1층)
9266,2019,20200918,영업,0,어린이집,아트어린이집,서울특별시 성북구 상월곡동 62
9267,2019,20200921,영업,0,어린이집,구립 송파파크센트럴 어린이집,서울특별시 송파구 거여동 234 e편한세상 송파 파크센트럴
9268,2019,20200924,영업,0,어린이집,당산2동 은솔어린이집,서울특별시 영등포구 당산동5가 4-13


In [141]:
fin_df.to_csv('data/어린이집_유치원 2017-2019(전처리, cp949).csv', encoding='cp949', index=False)

In [142]:
# 2020년 데이터
df_2020 = result_df[(result_df['인허가일자']<20210101) & (result_df['폐업일자']>=20200101) | (result_df['폐업일자']==0)].reset_index(drop=True)
df_2020['년'] = 2020
df_2020

,인허가일자,상세영업상태명,폐업일자,주소,사업장명,업태구분명,년
0,20200714,영업,0,서울특별시 마포구 대흥동 17-5 신촌그랑자이,신촌그랑어린이집,어린이집,2020
1,20200714,영업,0,서울특별시 은평구 응암동 626-53 백련산파크자이,구립한빛어린이집,어린이집,2020
2,20050707,영업,0,서울특별시 강서구 내발산동 746-0 사무엘어린이집,사무엘어린이집,어린이집,2020
3,20130726,영업,0,서울특별시 영등포구 문래동3가 55-18 영시티,하람어린이집,어린이집,2020
4,20200716,영업,0,서울특별시 은평구 응암동 36 녹번역e편한세상캐슬아파트,구립응암행복어린이집,어린이집,2020
...,...,...,...,...,...,...,...
2889,20081203,폐업,20200312,서울특별시 노원구 상계동 720 주공아파트 6단지내,영산유치원,유치원,2020
2890,20051021,폐업,20200729,서울특별시 관악구 신림동 1433-55,신지유치원,유치원,2020
2891,20161025,폐업,20200803,서울특별시 성북구 돈암동 609,우촌유치원,유치원,2020
2892,20070904,폐업,20200812,서울특별시 노원구 중계동 511-2 상원유치원,상원유치원,유치원,2020


In [143]:
# 영업, 폐업 구분
## open_df : 2020년에 영업
## closed_df : 2020년에 폐업
open_df = df_2020[(df_2020['폐업일자'] >= 20210101) | (df_2020['폐업일자'] == 0)].reset_index(drop=True)
closed_df = df_2020[(df_2020['폐업일자'] < 20210101)&(df_2020['폐업일자'] != 0)].reset_index(drop=True)
print(len(df_2020), len(open_df)+len(closed_df))
closed_df

2894 2894


,인허가일자,상세영업상태명,폐업일자,주소,사업장명,업태구분명,년
0,20130227,폐업,20200714,서울특별시 구로구 구로동 43-4 다솜아파트 110호 111호,다솜어린이집,어린이집,2020
1,20130726,폐업,20200715,서울특별시 구로구 구로동 1258 중앙유통단지 업무B동 1호,하람어린이집,어린이집,2020
2,20050329,폐업,20200716,서울특별시 노원구 공릉동 81 태강아파트관리동,미광어린이집,어린이집,2020
3,20040604,폐업,20200722,서울특별시 은평구 역촌동 66-11,구립역촌2동어린이집,어린이집,2020
4,20050630,폐업,20200602,서울특별시 관악구 봉천동 1712번지,소슬어린이집,어린이집,2020
...,...,...,...,...,...,...,...
196,20081203,폐업,20200312,서울특별시 노원구 상계동 720 주공아파트 6단지내,영산유치원,유치원,2020
197,20051021,폐업,20200729,서울특별시 관악구 신림동 1433-55,신지유치원,유치원,2020
198,20161025,폐업,20200803,서울특별시 성북구 돈암동 609,우촌유치원,유치원,2020
199,20070904,폐업,20200812,서울특별시 노원구 중계동 511-2 상원유치원,상원유치원,유치원,2020


In [144]:
open_df['폐업일자'] = 0
open_df['상세영업상태명'] = '영업'
open_df

,인허가일자,상세영업상태명,폐업일자,주소,사업장명,업태구분명,년
0,20200714,영업,0,서울특별시 마포구 대흥동 17-5 신촌그랑자이,신촌그랑어린이집,어린이집,2020
1,20200714,영업,0,서울특별시 은평구 응암동 626-53 백련산파크자이,구립한빛어린이집,어린이집,2020
2,20050707,영업,0,서울특별시 강서구 내발산동 746-0 사무엘어린이집,사무엘어린이집,어린이집,2020
3,20130726,영업,0,서울특별시 영등포구 문래동3가 55-18 영시티,하람어린이집,어린이집,2020
4,20200716,영업,0,서울특별시 은평구 응암동 36 녹번역e편한세상캐슬아파트,구립응암행복어린이집,어린이집,2020
...,...,...,...,...,...,...,...
2688,20200408,영업,0,서울특별시 서초구 우면동 731,서울어진유치원,유치원,2020
2689,20181005,영업,0,서울특별시 서초구 양재동 101 양재초등학교,서울양재유치원,유치원,2020
2690,20020318,영업,0,서울특별시 서초구 잠원동 56-3 잠원한신아파트,하나유치원,유치원,2020
2691,20200812,영업,0,서울특별시 노원구 중계동 511-2 상원유치원,상원유치원,유치원,2020


In [145]:
fin_2020_df = pd.concat([open_df, closed_df], axis=0).reset_index(drop=True)
fin_2020_df.sort_values(by=['인허가일자'], axis=0, inplace=True)
fin_2020_df.reset_index(drop=True, inplace=True)

In [147]:
fin_2020_df = fin_2020_df[['년', '인허가일자', '상세영업상태명', '폐업일자', '업태구분명', '사업장명', '주소']]
fin_2020_df.to_csv('data/어린이집_유치원 2020(전처리, cp949).csv', encoding='cp949', index=False)

### 좌표추가된 데이터 전처리

In [7]:
data_2017 = pd.read_csv('data/어린이집_유치원 2017-2019(좌표추가, cp949).csv', encoding='cp949')
data_2020 = pd.read_csv('data/어린이집_유치원 2020(좌표추가, cp949).csv', encoding='cp949')

In [4]:
data_2017

,No,년,인허가일자,상세영업상태명,폐업일자,업태구분명,사업장명,주소,경도,위도,상태
0,1,2017,19950120,영업,0,어린이집,개미어린이집,서울특별시 금천구 독산동 993-30번지 개미어린이집,126.905875,37.473026,정좌표
1,2,2017,19950121,영업,0,어린이집,정심어린이집,서울특별시 금천구 독산동 938-5번지,126.907258,37.467753,정좌표
2,3,2017,19950122,영업,0,어린이집,독산1동어린이집,서울특별시 금천구 독산동 999-11번지 독산1동어린이집,126.890859,37.468222,정좌표
3,4,2017,19950123,영업,0,어린이집,새길어린이집,서울특별시 금천구 시흥동 산 13-8번지,126.907530,37.440138,인근좌표
4,5,2017,19950124,영업,0,어린이집,가산어린이집,서울특별시 금천구 가산동 602-71번지 가산어린이집상록경록당,126.884379,37.482694,정좌표
...,...,...,...,...,...,...,...,...,...,...,...
9265,9266,2019,20200915,영업,0,어린이집,지니어스어린이집,서울특별시 강서구 등촌동 637 서울시니어스가양타워(지상 1층),126.859227,37.556925,정좌표
9266,9267,2019,20200918,영업,0,어린이집,아트어린이집,서울특별시 성북구 상월곡동 62,127.044827,37.606058,정좌표
9267,9268,2019,20200921,영업,0,어린이집,구립 송파파크센트럴 어린이집,서울특별시 송파구 거여동 234 e편한세상 송파 파크센트럴,127.150589,37.493167,인근좌표
9268,9269,2019,20200924,영업,0,어린이집,당산2동 은솔어린이집,서울특별시 영등포구 당산동5가 4-13,126.903069,37.530103,정좌표


In [5]:
# 필요없는 컬럼 삭제, 저장
data_2017.drop(['No', '상태'], axis=1, inplace=True)
data_2017.to_csv('data/어린이집_유치원 2017-2019(전처리완료, cp949).csv', encoding='cp949', index=False)

In [8]:
data_2020

,No,년,인허가일자,상세영업상태명,폐업일자,업태구분명,사업장명,주소,경도,위도,상태
0,1,2020,19950120,영업,0,어린이집,개미어린이집,서울특별시 금천구 독산동 993-30번지 개미어린이집,126.905876,37.473026,정좌표
1,2,2020,19950122,영업,0,어린이집,독산1동어린이집,서울특별시 금천구 독산동 999-11번지 독산1동어린이집,126.890859,37.468222,정좌표
2,3,2020,19950123,영업,0,어린이집,새길어린이집,서울특별시 금천구 시흥동 산 13-8번지,126.907530,37.440138,인근좌표
3,4,2020,19950124,영업,0,어린이집,가산어린이집,서울특별시 금천구 가산동 602-71번지 가산어린이집상록경록당,126.884379,37.482694,정좌표
4,5,2020,19950126,영업,0,어린이집,옥계어린이집,서울특별시 금천구 독산동 1067-9번지 옥계어린이집,126.900344,37.460111,정좌표
...,...,...,...,...,...,...,...,...,...,...,...
2889,2890,2020,20200915,영업,0,어린이집,지니어스어린이집,서울특별시 강서구 등촌동 637 서울시니어스가양타워(지상 1층),126.859227,37.556925,정좌표
2890,2891,2020,20200918,영업,0,어린이집,아트어린이집,서울특별시 성북구 상월곡동 62,127.044827,37.606058,정좌표
2891,2892,2020,20200921,영업,0,어린이집,구립 송파파크센트럴 어린이집,서울특별시 송파구 거여동 234 e편한세상 송파 파크센트럴,127.150589,37.493167,인근좌표
2892,2893,2020,20200924,영업,0,어린이집,당산2동 은솔어린이집,서울특별시 영등포구 당산동5가 4-13,126.903069,37.530103,정좌표


In [9]:
# 필요없는 컬럼 삭제, 저장
data_2020.drop(['No', '상태'], axis=1, inplace=True)
data_2020.to_csv('data/어린이집_유치원 2020(전처리완료, cp949).csv', encoding='cp949', index=False)